# Ross Ice Shelf gravity inversion

# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz

from RIS_gravity_inversion import regional, synthetic, plotting, optimization
import RIS_gravity_inversion.inversion as inv
import RIS_gravity_inversion.utils as inv_utils

import copy
from antarctic_plots import utils, regions, maps, fetch, profile
import zarr
import harmonica as hm
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import verde as vd
import scipy as sp
import pvxarray
import rioxarray
import pygmt
import geopandas as gpd
import plotly.graph_objects as go
import optuna
from optuna.storages import JournalStorage, JournalFileStorage
import warnings
import itertools

import os

# make beep sound when notify is called at end of long-running cells
from IPython.display import Audio

wave = np.sin(2 * np.pi * 400 * np.arange(10000 * 1) / 10000)
notify = Audio(wave, rate=10000, autoplay=True)

In [ ]:
# from pygmt import session_management

# session_management.end()
# session_management.begin()

# Set region and parameters

In [ ]:
# polygon = regions.draw_region()

In [ ]:
# starting_region = utils.polygon_to_region(polygon)

In [ ]:
starting_region = [-580e3, 420e3, -1420e3, -420e3]

inversion_region = starting_region
buffer_region = vd.pad_region(starting_region, 40e3)
# inversion_region, buffer_region = utils.alter_region(
#     starting_region=starting_region,
#     zoom=30e3,
#     # n_shift=0e3,
#     # w_shift=-0e3,
#     buffer=30e3,
# )

# set densities
air_density = 1
ice_density = 917  # +/- 5, from Griggs and Bamber 2009/2011
water_density = 1024  # +/- 5, from Griggs and Bamber 2009/2011
sediment_density = 2300  # will likely change this

# set layer parameters
layer_spacing = 5e3
registration = "g"

# set gravity observation parameters
grav_observation_height = 1e3
grav_spacing = 5e3

print(f"layer spacing: {layer_spacing}m")
print(f"registration: {registration}")
print("inversion region = ", inversion_region)
print("buffer region = ", buffer_region)

# Load data

## topographic layers
* Surface, Icebase, Bed

In [ ]:
# SURFACE
fname = "inversion_layers/bedmachine_surface.zarr"
# fname= "inversion_layers/bedmap2_surface.zarr"
surface_full = xr.open_zarr(fname).surface

# ICEBASE
fname = "inversion_layers/bedmachine_icebase.zarr"
# fname =  "inversion_layers/bedmap2_icebase.zarr"
icebase_full = xr.open_zarr(fname).icebase

# BED
fname = "inversion_layers/bedmachine_bed.zarr"
bedmachine_bed_full = xr.open_zarr(fname).bed

fname = "inversion_layers/bedmap2_bed.zarr"
bedmap_bed_full = xr.open_zarr(fname).bed

# bedmachine outside RIS, riggs inside
if layer_spacing == 5e3:
    fname = "inversion_layers/bed_from_constraints_5k.zarr"
elif layer_spacing == 10e3:
    fname = "inversion_layers/bed_from_constraints_10k.zarr"
else:
    raise ValueError("No bed grids available at specified spacing!")

bed_full = xr.open_zarr(fname).bed

# bedmachine outside RIS, bedmap2 inside RIS, and RIGGS points within RIS
# bed = "inversion_layers/bedmap_bedmachine_merge.zarr"

# BASEMENT
# Ross Embayment basement merged to bedmachine bed
# basement = "inversion_layers/bed_basement_merged.zarr"
# Ross Embayment basement merged to ANTASed basement
# basement = "inversion_layers/ANTASed_magnetics_merged_basement.zarr"
# basement = xr.open_zarr(basement).basement

grids = [
    surface_full,
    icebase_full,
    bed_full,
    bedmap_bed_full,
    bedmachine_bed_full,
]

for g in grids:
    print(utils.get_grid_info(g))

### resample to same spacing / region

In [ ]:
surface = fetch.resample_grid(
    surface_full,
    spacing=1e3,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
icebase = fetch.resample_grid(
    icebase_full,
    spacing=1e3,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
icebase_layer_spacing = fetch.resample_grid(
    icebase_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
bed = fetch.resample_grid(
    bed_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
bedmap_bed = fetch.resample_grid(
    bedmap_bed_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
bedmachine_bed = fetch.resample_grid(
    bedmachine_bed_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)

# ensure layers never cross
icebase = xr.where(icebase > surface, surface, icebase)
bed = xr.where(
    bed > icebase_layer_spacing,
    icebase_layer_spacing,
    bed,
)

grids = [
    surface,
    icebase,
    icebase_layer_spacing,
    bed,
    bedmap_bed,
    bedmachine_bed,
]

for g in grids:
    print(utils.get_grid_info(g))

# set conventional names
# surface = surface.rename({"x": "easting", "y": "northing"})
# icebase = icebase.rename({"x": "easting", "y": "northing"})
# bed = bed.rename({"x": "easting", "y": "northing"})
bedmachine_bed = bedmachine_bed.rename({"x": "easting", "y": "northing"})
bedmap_bed = bedmap_bed.rename({"x": "easting", "y": "northing"})

## bed constraints
from notebook `RIS_bathymetry_data.ipynb`

In [ ]:
# constraints_fname = "data/bedmap1.csv.gz"
if layer_spacing == 5e3:
    constraints_fname = "data/bed_constraints_5k.csv.gz"
elif layer_spacing == 10e3:
    constraints_fname = "data/bed_constraints_10k.csv.gz"
else:
    raise ValueError("No constraint point files available at specified spacing!")

constraints = pd.read_csv(
    constraints_fname,
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

# constraints = utils.block_reduce(
#     constraints,
#     np.median,
#     spacing = layer_spacing,
#     input_data_names = ["z_ellipsoidal"],
#     center_coordinates=True,
#     adjust="region",
#     drop_coords=False,
# )

# ensure all points are within buffer region
constraints = utils.points_inside_region(
    constraints, buffer_region, names=("easting", "northing")
)

print(f"bathymetry control points:{len(constraints)}")
constraints.describe()

In [ ]:
maps.plot_grd(
    bed,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    points_style="c0.05c",
)

## gravity data

github repository [RIS_grav_bath_data](https://github.com/mdtanker/RIS_grav_bath_data), contains a compilation of gravity data and bed elevation constraints for the Ross Ice Shelf region. Load the data into a pandas dataframe with the below cell. 

> **Note:** this repo is currently private, so it requires a github authentication token!

In [ ]:
# load gravity data
# grav_grid_full = xr.open_zarr("inversion_layers/rosetta_eq_source_1k_up_continued.zarr").load()
grav_grid_full = xr.open_zarr(
    "inversion_layers/rosetta_1k_up_continued_20k_masked.zarr"
).load()

grav_grid_full.Gobs.plot()

In [ ]:
grav_grid = inv_utils.filter_grid(
    grav_grid_full.Gobs,
    50e3,
    filt_type="lowpass",
)
# grav_grid = inv_utils.filter_grid(
#     grav_grid,
#     5e3,
#     filt_type="lowpass",
#     # change_spacing=True,
# )
# grav_grid = grav_grid_full.Gobs

print(utils.get_grid_info(grav_grid))

grav_grid.plot()

In [ ]:
grav = vd.grid_to_table(grav_grid)
grav = grav[grav.Gobs.notna()]
grav["upward"] = 1000
grav.describe()

In [ ]:
grav = inv_utils.prep_grav_data(
    grav,
    region=inversion_region,
    input_grav_name="Gobs",
    input_coord_names=("easting", "northing", "upward"),
)
grav.describe()

In [ ]:
# grav_fname = "inversion_layers/ross_ice_shelf_gravity_data.csv.gz"
# # grav_fname = fetch_private_github_file(grav, output_dir="../inversion_layers/")
# # print(grav_fname)

# grav = pd.read_csv(
#     grav_fname,
#     sep=",",
#     header="infer",
#     index_col=None,
#     compression="gzip",
# )

# grav = inv_utils.prep_grav_data(
#     grav,
#     region=inversion_region,
#     input_grav_name="free_air_anomaly",
#     input_coord_names=("x", "y", "ellipsoidal_elevation"),
# )
# grav.describe()

# Plot input data

In [ ]:
fig = plotting.plot_data_and_layers(
    [surface, icebase, bed],
    ["surface", "icebase", "bed"],
    grav.set_index(["northing", "easting"]).to_xarray().Gobs,
    inversion_region=inversion_region,
    buffer_region=buffer_region,
    # points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    points_style="c0.05c",
    robust=True,
)
fig.show()

In [ ]:
# fig = go.Figure(
#     data=[
#         go.Surface(
#             z=surface.values,
#             colorscale="Blues",
#             opacity=0.8,
#             colorbar_x=1,
#         ),
#         go.Surface(
#             z=icebase.values,
#             colorscale="Greens",
#             opacity=0.4,
#             colorbar_x=1.2,
#         ),
#         go.Surface(
#             z=bed.values,
#             colorscale="Electric",
#             colorbar_x=1.4,
#         ),
#     ],
# )


# fig.update_traces(
#     contours_z=dict(
#         show=True,
#         usecolormap=True,
#         highlightcolor="limegreen",
#         project_z=True,
#     ),
# )

# camera = dict(eye=dict(x=0.5, y=0.5, z=0.5))
# fig.update_layout(
#     # title=title,
#     scene_camera=camera,
# )

# fig.show()

In [ ]:
# grav = misfit

# Bouguer corrections

### Surface prisms

In [ ]:
# create prisms around surface
# positive densities above reference, negative below
density = ice_density - air_density
print(f"Surface density contrast: {density}kgm-3")

surface_prisms = inv_utils.grids_to_prisms(
    surface=surface,
    reference=surface.values.mean(),
    density=xr.where(surface >= surface.values.mean(), density, -density),
    input_coord_names=["x", "y"],
)

# plotting.show_prism_layers(
#     [surface_prisms],
#     cmap="viridis",
#     color_by="density",
#     # color_by="thickness",
#     zscale=100,
#     clip_box=False,
#     log_scale=False,
# )

### Surface gravity

In [ ]:
# calculate gravity of surface
surface_grav_grid, surface_grav_df = inv_utils.forward_grav_of_prismlayer(
    [surface_prisms],
    grav[grav.Gobs.notna()],
    names=["surface_prisms"],
    progressbar=True,
    plot=False,
)

grav["surface_forward"] = surface_grav_df.forward_total

grav.describe()

In [ ]:
# grid the results
surface_forward = grav.set_index(["northing", "easting"]).to_xarray().surface_forward
surface_forward.plot()

### Icebase prisms

In [ ]:
# create prisms around icebase
# positive densities above reference, negative below
density = water_density - ice_density
print(f"Icebase density contrast: {density}kgm-3")

icebase_prisms = inv_utils.grids_to_prisms(
    surface=icebase,
    reference=icebase.values.mean(),
    density=xr.where(icebase >= icebase.values.mean(), density, -density),
    input_coord_names=["x", "y"],
)

# plotting.show_prism_layers(
#     [icebase_prisms],
#     cmap="viridis",
#     color_by="density",
#     # color_by="thickness",
#     zscale=100,
#     clip_box=False,
#     log_scale=False,
# )

### Icebase gravity

In [ ]:
# calculate gravity of icebase
icebase_grav_grid, icebase_grav_df = inv_utils.forward_grav_of_prismlayer(
    [icebase_prisms],
    grav[grav.Gobs.notna()],
    names=["icebase_prisms"],
    progressbar=True,
    plot=False,
)

grav["icebase_forward"] = icebase_grav_df.forward_total


grav.describe()

In [ ]:
# grid the results
icebase_forward = grav.set_index(["northing", "easting"]).to_xarray().icebase_forward

icebase_forward.plot()

## Correct the observed gravity

In [ ]:
grav["Gobs_corr"] = grav.Gobs - grav.surface_forward - grav.icebase_forward
grav.describe()

### Save and load results

In [ ]:
grav.to_csv(
    "data/RIS_bouguer_corrections_50k_filt_mask.csv.gz",
    # "data/RIS_bouguer_corrections_50k_filt.csv.gz",
    sep=",",
    na_rep="",
    header=True,
    index=False,
    encoding="utf-8",
    compression="gzip",
)

In [ ]:
grav = pd.read_csv(
    "data/RIS_bouguer_corrections_50k_filt_mask.csv.gz",
    # "data/RIS_bouguer_corrections_50k_filt.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

grav.describe()

In [ ]:
plotting.corrections_plotting(
    grav,
    [
        "surface_forward",
        "icebase_forward",
        "Gobs_corr",
        "Gobs",
    ],
    titles=[
        "Surface forward gravity",
        "Icebase forward gravity",
        "Corrected gravity",
        "Observed gravity",
    ],
    buffer_region=buffer_region,
    inversion_region=inversion_region,
    robust=True,
    # points = constraints,
    # points_style = "c0.05c",
)

# Gravity misfit

## Bed prisms

In [ ]:
# create prisms around bed
# positive densities above reference, negative below
density = sediment_density - water_density
print(f"Bed density contrast: {density}kgm-3")

bed_prisms = inv_utils.grids_to_prisms(
    surface=bed,
    reference=bed.values.mean(),
    density=xr.where(bed >= bed.values.mean(), density, -density),
    input_coord_names=["x", "y"],
)

# plotting.show_prism_layers(
#     [bed_prisms],
#     cmap="viridis",
#     color_by="density",
#     # color_by="thickness",
#     zscale=100,
#     clip_box=False,
#     log_scale=False,
# )

## Bed gravity

In [ ]:
# calculate gravity of icebase
bed_grav_grid, bed_grav_df = inv_utils.forward_grav_of_prismlayer(
    [bed_prisms],
    grav[grav.Gobs.notna()],
    names=["bed_prisms"],
    progressbar=True,
    plot=False,
)

grav["bed_forward"] = bed_grav_df.forward_total

grav.describe()

In [ ]:
# grid the results
bed_forward = grav.set_index(["northing", "easting"]).to_xarray().bed_forward
bed_forward.plot()

In [ ]:
grav.to_csv(
    "data/RIS_bouguer_corrections_50k_filt_mask.csv.gz",
    # "data/RIS_bouguer_corrections_50k_filt.csv.gz",
    sep=",",
    na_rep="",
    header=True,
    index=False,
    encoding="utf-8",
    compression="gzip",
)

In [ ]:
grav = pd.read_csv(
    "data/RIS_bouguer_corrections_50k_filt_mask.csv.gz",
    # "data/RIS_bouguer_corrections_50k_filt.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

grav

## misfit

In [ ]:
misfit = inv.misfit(
    input_grav=grav,
    input_forward_column="bed_forward",
    input_grav_column="Gobs_corr",
    constraints=constraints,
)
misfit.describe()

### Save and load results

In [ ]:
misfit.set_index(["northing", "easting"]).to_xarray().misfit.plot(robust=True)

In [ ]:
fig = plotting.misfit_plotting(
    misfit,
    plot_type="pygmt",
    input_forward_column="bed_forward",
    input_grav_column="Gobs_corr_shift",
    robust=True,
)
fig.show()

# Regional - Residual seperation

The `corrected observed gravity` is the combination of the signal from the density contrasts of `bathymetry` and `basement`. Both of these signals contain long-wavelength components, since both surface have long-wavelength topographic features. The `bathymetry` signal also contains short-wavelength components. Deeper in the crust, further away from the gravity observation points, these short-wavelength components decrease in amplitude. This means the majority the short-wavelength signal in the `corrected observed gravity` is resulting from the density contrast of the `bathymetry`. This is the signal we want to isolate and input into the inversion. 

To do this, we need to seperate the gravity misfit into a `regional` component and a `residual` component. This `regional` field should theoretically contain every gravity signal other than the that resulting from the `bathymetry` surface. Additionaly, this `residual` component should theoretically represent the gravity anomaly due to the difference between the true `bathymetry` and the low-resolution `bathymetry`. 

If we fail to accurately remove the `regional` signal, the resulting `bathymetry` after the inversion will contain long-wavelength topography, resulting from the `regional` gravity field.  

In [ ]:
# choose the regional method:
# regional_method = "filter"
# regional_method = "trend"
regional_method = "constraints"
# regional_method = "eq_sources"

# set the parameter:
# contraints: 1 results in pedastals, 0 is good,
param = 0.25

df_anomalies = regional.regional_seperation(
    regional_method=regional_method,
    filter=f"g{param}",
    # filter=param,
    trend=param,
    tension_factor=param,
    eq_sources=param,
    input_grav=misfit,
    input_forward_column="bed_forward",
    input_grav_column="Gobs_corr_shift",
    grav_spacing=grav_spacing,
    inversion_region=inversion_region,
    constraints=constraints,
    block_size=layer_spacing * 10,
    # grid_method="verde",
    # block_reduce=True,
    #     delayed=False,
)

anom_grids = plotting.anomalies_plotting(
    df_anomalies,
    region=inversion_region,
    grav_spacing=layer_spacing,
    title=f" Method: {regional_method}, parameter: {param}",
    # constraints=constraints,
    input_forward_column="bed_forward",
    input_grav_column="Gobs_corr_shift",
    robust=True,
    # shp_mask = gpd.read_file("plotting/RIS_outline.shp").buffer(-20e3)
)

### Save and load results

In [ ]:
df_anomalies.to_csv(
    "data/RIS_residual_50k_filt_mask.csv.gz",
    # "data/RIS_residual_50k_filt.csv.gz",
    sep=",",
    na_rep="",
    header=True,
    index=False,
    encoding="utf-8",
    compression="gzip",
)

In [ ]:
df_anomalies = pd.read_csv(
    "data/RIS_residual_50k_filt_mask.csv.gz",
    # "data/RIS_residual_50k_filt.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

df_anomalies

In [ ]:
df_anomalies.describe()

# Plot Inputs

In [ ]:
plotting.inputs_plotting(
    df_anomalies,
    bed,
    buffer_region=buffer_region,
    inversion_region=inversion_region,
    robust=True,
    # points = constraints,
    # points_style = "c0.05c",
)

# Constraints grid


In [ ]:
# starting_prisms = copy.deepcopy(bed_prisms)

# weights, min_dist = inv_utils.constraints_grid(
#     constraints,
#     starting_prisms,
#     inner_bound=5e3,
#     outer_bound=20e3,
#     low=0,
#     high=1,
#     region=inversion_region,
#     interp_type="spline",
#     efficient_interp=True,
#     # tension_factor = .9,
#     # plot = True,
# )

In [ ]:
# starting_prisms["weights"] = weights
# starting_prisms["min_dist"] = min_dist
# starting_prisms

In [ ]:
# starting_prisms.weights.plot(cmap="Blues")

In [ ]:
# starting_prisms.min_dist.plot(cmap="Blues")

### Save to Zarr

In [ ]:
# enc = {x: {"compressor": zarr.Blosc()} for x in starting_prisms}
# starting_prisms.to_zarr(
#     f"inversion_layers/RIS_starting_prisms_{int(layer_spacing)}k_constraints.zarr",
#     encoding=enc,
#     mode="w",
# )

In [ ]:
starting_prisms = xr.open_zarr(
    f"inversion_layers/RIS_starting_prisms_{int(layer_spacing)}k_constraints.zarr"
)

In [ ]:
fig = maps.plot_grd(
    starting_prisms.weights,
    fig_height=10,
    title="Weighting grid",
    cbar_label="weights",
    cmap="dense",
    # points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    points_style="c0.05c",
)
fig = maps.plot_grd(
    starting_prisms.min_dist,
    fig=fig,
    origin_shift="xshift",
    title="Minumum distance to constraints",
    cbar_label="m",
    cmap="viridis",
    # points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    points_style="c0.05c",
)

fig.show()

# Geo inversion

In [ ]:
no_weights_results = inv.geo_inversion(
    input_grav=df_anomalies,
    input_grav_column="Gobs_corr_shift",
    prism_layer=starting_prisms,
    # apply_weights = True,
    max_iterations=2,
    l2_norm_tolerance=0.2,  # 0.01, #.2,
    delta_l2_norm_tolerance=0.8,  # =.1, #.8,
    perc_increase_limit=0.2,  # 1000, #.2,
    upper_confining_layer=icebase_layer_spacing,
    deriv_type="annulus",
    # deriv_type = "prisms",
    solver_type="scipy least squares",
    # solver_type = "gauss newton",
    # solver_type = "verde least squares",
    solver_damping=0.1,
    # solver_damping = 1000
    # save_results=True,
    # fname_prefix= "results/no_weights_mask",
)

In [ ]:
# prism_results, grav_results, params, elapsed_time = no_weights_results
prism_results = "results/no_weights_mask_prism_results.csv.gz"
grav_results = "results/no_weights_mask_gravity_results.csv.gz"
params = "results/no_weights_mask_parameters.npy"

no_weights_grids = plotting.plot_inversion_results(
    grav_results,
    prism_results,
    params,
    grav_region=inversion_region,
    grav_spacing=layer_spacing,
    constraints=constraints,
    # plot_iter_results=False
    # plot_topo_results=False,
    plot_grav_results=False,
    iters_to_plot=2,
    shp_mask="plotting/RIS_outline.shp",
    misfit_cmap_perc=0.2,
    topo_cmap_perc=0.8,
    corrections_cmap_perc=0.2,
)
no_weights_final_topo = no_weights_grids[1][-1]
no_weights_topo_diff = (
    bed.rename({"x": "easting", "y": "northing"}) - no_weights_final_topo
)

In [ ]:
dif = no_weights_topo_diff
dif = utils.mask_from_shp(
    "plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    dif,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="vik+h0",
    points_style="c0.05c",
    # grd2cpt=True,
    hist=True,
    title="total inversion correction, no weights",
    cbar_unit="m",
    robust=True,
)
fig.show()

In [ ]:
fig = maps.plot_grd(
    no_weights_final_topo,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="rain",
    reverse_cpt=True,
    points_style="c0.05c",
    grd2cpt=True,
    cpt_lims=[-1200, 0],
    hist=True,
    title="final topography, no weights",
    cbar_unit="m",
    # robust=True,
)
fig.show()

In [ ]:
dif = no_weights_final_topo - bedmachine_bed
dif = utils.mask_from_shp(
    "plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    dif,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="vik+h0",
    points_style="c0.05c",
    # grd2cpt=True,
    # cpt_lims=[-200, 200],
    hist=True,
    title="difference with bedmachine, no weights",
    cbar_unit="m",
    robust=True,
)
fig.show()

In [ ]:
dif = no_weights_final_topo - bedmap_bed
dif = utils.mask_from_shp(
    "plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    dif,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="vik+h0",
    points_style="c0.05c",
    # grd2cpt=True,
    # cpt_lims=[-200, 200],
    hist=True,
    title="difference with bedmap, no weights",
    cbar_unit="m",
    robust=True,
)
fig.show()

In [ ]:
with_weights_results = inv.geo_inversion(
    input_grav=df_anomalies,
    input_grav_column="Gobs_corr_shift",
    prism_layer=starting_prisms,
    apply_weights=True,
    max_iterations=20,
    l2_norm_tolerance=0.2,  # 0.01, #.2,
    delta_l2_norm_tolerance=0.8,  # .1, #.8,
    perc_increase_limit=0.2,  # 1000, #.2,
    upper_confining_layer=icebase_layer_spacing,
    # max_layer_change_per_iter = 0.0,
    deriv_type="annulus",
    # deriv_type = "prisms",
    solver_type="scipy least squares",
    # solver_type = "gauss newton",
    # solver_type = "verde least squares",
    solver_damping=0.1,
    # solver_damping = 1000
    save_results=True,
    fname_prefix="results/weights_mask",
)

In [ ]:
# prism_results, grav_results, params, elapsed_time = with_weights_results
prism_results = "results/weights_mask_prism_results.csv.gz"
grav_results = "results/weights_mask_gravity_results.csv.gz"
params = "results/weights_mask_parameters.npy"

with_weights_grids = plotting.plot_inversion_results(
    grav_results,
    prism_results,
    params,
    grav_region=inversion_region,
    grav_spacing=layer_spacing,
    constraints=constraints,
    # plot_iter_results=False
    # plot_topo_results=False,
    plot_grav_results=False,
    iters_to_plot=2,
    shp_mask="plotting/RIS_outline.shp",
    misfit_cmap_perc=0.2,
    topo_cmap_perc=0.8,
    corrections_cmap_perc=0.2,
)
with_weights_final_topo = with_weights_grids[1][-1]
with_weights_topo_diff = (
    bed.rename({"x": "easting", "y": "northing"}) - with_weights_final_topo
)

In [ ]:
maps.plot_grd(
    with_weights_final_topo,
    region=utils.alter_region(regions.minna_bluff, zoom=-200e3)[0],
    hist=True,
    cmap="viridis",
    # reverse_cpt=True,
    coast=True,
    inset=True,
    robust=True,
    cpt_lims=[-1000, 0],
)

In [ ]:
dif = with_weights_topo_diff
dif = utils.mask_from_shp(
    "plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    dif,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="vik+h0",
    points_style="c0.05c",
    # grd2cpt=True,
    hist=True,
    title="total inversion correction, with weights",
    cbar_unit="m",
    robust=True,
)
fig.show()

In [ ]:
fig = maps.plot_grd(
    with_weights_final_topo,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="rain",
    reverse_cpt=True,
    points_style="c0.05c",
    grd2cpt=True,
    cpt_lims=[-1200, 0],
    hist=True,
    title="final topography, with weights",
    cbar_unit="m",
    # robust=True,
)
fig.show()

In [ ]:
dif = with_weights_final_topo - bedmachine_bed
dif = utils.mask_from_shp(
    "plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    dif,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="vik+h0",
    points_style="c0.05c",
    # grd2cpt=True,
    # cpt_lims=[-200, 200],
    hist=True,
    title="difference with bedmachine, with weights",
    cbar_unit="m",
    robust=True,
)
fig.show()

In [ ]:
# function to find the closest odd integer to a given number
def closest_odd(x):
    return int(np.ceil(x / 2.0) * 2 - 1)

In [ ]:
dif = with_weights_final_topo - bedmap_bed
dif = utils.mask_from_shp(
    "plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    dif,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="vik+h0",
    points_style="c0.05c",
    # grd2cpt=True,
    # cpt_lims=[-200, 200],
    hist=True,
    title="difference with bedmap, with weights",
    cbar_unit="m",
    robust=True,
)
fig.show()

In [ ]:
dif = no_weights_final_topo - with_weights_final_topo
dif = utils.mask_from_shp(
    "plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    dif,
    points=constraints.rename(columns={"easting": "x", "northing": "y"}),
    cmap="vik+h0",
    points_style="c0.05c",
    # grd2cpt=True,
    # cpt_lims=[-100, 100],
    hist=True,
    title="difference with and without weights",
    cbar_unit="m",
    robust=True,
)
fig.show()

In [ ]:
# from ipyfastscape import TopoViz3d

# ds2 = da2.to_dataset()

# app = TopoViz3d(ds2, elevation_var="z")

# app.show()

# Density inversion

In [ ]:
# density_inversion(
# density_layer='bathymetry',
# plot=True
# )
# 10mins, 200km zoom/5k, 3.5mins 400km zoom/5k (113x113 prisms, 33x33 grav)
# notify

# Cross sections

In [ ]:
# lines = profile.draw_lines()

In [ ]:
# get clicked points
# df = utils.shapes_to_df(lines)
# # extract first and last
# a = (df.x.iloc[0], df.y.iloc[0])
# b = (df.x.iloc[-1], df.y.iloc[-1])

In [ ]:
# # extract layers for profile plot
# names = []
# topo_grids = []
# forward_grav_grids = []
# for k, v in layers.items():
#     names.append(k)
#     topo_grids.append(v['grid'])
#     forward_grav_grids.append(v['forward_grav_grid'])

# layers_dict = profile.make_data_dict(
#     names = names,
#     grids = topo_grids,
#     colors = ["lightskyblue","darkblue","lightbrown","chocolate", "darkolivegreen"],
# )

# data_dict = profile.make_data_dict(
#     names = names,
#     grids = forward_grav_grids,
#     colors = ["red","blue","purple","black", "magenta"]
#     )

# # add dictionary entry for total forward gravity
# data_dict["total"] = {}
# data_dict["total"]["name"] = "total"
# data_dict["total"]["grid"] = forward_grav_total
# data_dict["total"]["color"] = "forestgreen"

# a = (0, -641024)
# b = (0, -1363968)

# c = (-50e3, -750e3)#(-200e3, -925e3)
# d = (50e3, -750e3)#(200e3, -925e3)

In [ ]:
# profile.plot_profile(
#     "points",
#     start=a,
#     stop=b,
#     num=1000,
#     add_map=True,
#     subplot_orientation='horizontal',
#     data_dict=data_dict,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )

In [ ]:
# profile.plot_profile(
#     "points",
#     start=c,
#     stop=d,
#     num=1000,
#     add_map=True,
#     subplot_orientation='vertical',
#     data_dict=data_dict,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     inset_width=.15,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )

In [ ]:
# anomalies_dict = profile.make_data_dict(
#     names = ["Observed", "Bouguer", "Forward total", "Misfit", "Regional misfit", "Residual misfit"],
#     grids = anom_grids,
#     colors = ["black", "turquoise", "red", "purple", "forestgreen", "blue"]
#     )

# keys = ["Observed", "Bouguer"] # The keys you want
# subset0 = dict((k, anomalies_dict[k]) for k in keys if k in anomalies_dict)

# keys = ["Bouguer", "Forward total", "Misfit",] # The keys you want
# subset1 = dict((k, anomalies_dict[k]) for k in keys if k in anomalies_dict)

# keys = ["Misfit", "Regional misfit", "Residual misfit"] # The keys you want
# subset2 = dict((k, anomalies_dict[k]) for k in keys if k in anomalies_dict)

# a = (0, -641024)
# b = (0, -1363968)

# c = (-50e3, -750e3)#(-200e3, -925e3)
# d = (50e3, -750e3)#(200e3, -925e3)

In [ ]:
# profile.plot_profile(
#     "points",
#     start=a,
#     stop=b,
#     num=1000,
#     add_map=True,
#     subplot_orientation='horizontal',
#     data_dict=subset0,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )

# profile.plot_profile(
#     "points",
#     start=a,
#     stop=b,
#     num=1000,
#     add_map=True,
#     subplot_orientation='horizontal',
#     data_dict=subset1,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )

# profile.plot_profile(
#     "points",
#     start=a,
#     stop=b,
#     num=1000,
#     add_map=True,
#     subplot_orientation='horizontal',
#     data_dict=subset2,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )

# just the data
# profile.plot_data(
#     "points",
#     start=a,
#     stop=b,
#     num=1000,
#     data_dict=subset2,
#     share_yaxis=True,
# )

In [ ]:
# profile.plot_profile(
#     "points",
#     start=c,
#     stop=d,
#     num=1000,
#     add_map=True,
#     subplot_orientation='vertical',
#     data_dict=subset0,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     inset_width=.15,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )

# profile.plot_profile(
#     "points",
#     start=c,
#     stop=d,
#     num=1000,
#     add_map=True,
#     subplot_orientation='vertical',
#     data_dict=subset1,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     inset_width=.15,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )

# profile.plot_profile(
#     "points",
#     start=c,
#     stop=d,
#     num=1000,
#     add_map=True,
#     subplot_orientation='vertical',
#     data_dict=subset2,
#     layers_dict=layers_dict,
#     share_yaxis=True,
#     inset_width=.15,
#     map_points=constraints_RIS_df,
#     data_height=5,
# )